In [1]:
# Load libraries
import numpy as np
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.losses import MeanAbsoluteError
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.optimizers import Adam

In [2]:
data = pd.read_csv('MF_data.csv')
data

Company        Time  Total Assets  Total Liabilities  Total Equity  \
0        DVA  2019-12-01       17311.4            13811.8        3499.6   
1        CVS  2012-12-01       66221.0            28568.0       37653.0   
2        NEM  2014-12-01       24916.0            11827.0       13089.0   
3       TTWO  2019-03-01        4243.1             2202.5        2040.6   
4        ADS  2018-12-01       30387.7            28055.6        2332.1   
5         MU  2011-09-01       14752.0             4900.0        9852.0   
6        ALK  2015-12-01        6530.0             4119.0        2411.0   
7        KMX  2019-02-01       18717.9            15360.8        3357.0   
8      MRWSY  2011-01-01       14493.5             5907.3        8586.1   
9        ETR  2018-12-01       48275.1            39211.4        9063.7   
10      TTWO  2012-03-01        1149.4              553.7         595.7   
11        WU  2010-12-01        7929.2             7346.5         582.7   
12       TDS  2017-12-01        9295.0             4402.0        4893.0   
13       CMC  2011-08-01        3683.1             2522.5        1160.6   
14       AOS  2017-12-01        3197.4             1552.5        1644.9   
15       XOM  2013-12-01      346808.0           166313.0      180495.0   
16       IFF  2017-12-01        4598.9             2909.6        1689.3   
17     MRWSY  2019-02-01       13951.8             8294.4        5657.4   
18       LUV  2017-12-01       25110.0            15469.0        9641.0   
19       CTL  2018-12-01       70256.0            50428.0       19828.0   
20      FAST  2010-12-01        1468.3              185.8        1282.5   
21        WU  2017-12-01        9231.4             9722.8        -491.4   
22       GPC  2011-12-01        6202.8             3449.2        2753.6   
23        WM  2019-12-01       27743.0            20673.0        7070.0   
24      GPRE  2013-12-01        1532.0              986.7         545.4   
25       LOW  2013-02-01       32666.0            18809.0       13857.0   
26       IFF  2010-12-01        2872.5             1869.3        1003.2   
27      ULTA  2016-01-01        2230.9              788.0        1442.9   
28       SWK  2015-01-01       15849.1             9337.2        6511.9   
29       APD  2010-09-01       13505.9             7808.3        5697.6   
...      ...         ...           ...                ...           ...   
4970     BLK  2012-12-01      200451.0           174834.0       25617.0   
4971     TFX  2017-12-01        6181.5             3751.0        2430.5   
4972   CPYYY  2019-12-01       24048.5            21670.7        2377.8   
4973    ALGN  2011-12-01         649.3              158.5         490.8   
4974    SOHU  2012-12-01        2082.6              705.6        1377.0   
4975    FAST  2017-12-01        2910.5              813.6        2096.9   
4976     ETR  2015-12-01       44647.7            35072.7        9575.0   
4977      HD  2019-02-01       44003.0            45881.0       -1878.0   
4978     CNK  2013-12-01        4144.2             3041.7        1102.4   
4979     CVX  2016-12-01      260078.0           113356.0      146722.0   
4980     ALB  2010-12-01        3068.1             1592.3        1475.7   
4981    TSCO  2014-12-01        2034.6              741.0        1293.6   
4982     PEG  2018-12-01       45326.0            30949.0       14377.0   
4983     GTN  2017-12-01        3260.9             2268.0         992.9   
4984     AAP  2017-12-01        8482.3             5067.1        3415.2   
4985      NI  2010-12-01       19913.4            15015.9        4897.5   
4986     SHW  2015-12-01        5778.9             4911.0         867.9   
4987     DOV  2013-12-01       10855.2             5477.8        5377.4   
4988     ALK  2016-12-01        9962.0             7031.0        2931.0   
4989      LH  2015-12-01       14104.7             9144.7        4960.0   
4990     ITW  2019-12-01       15068.0            12038.0        3030.0   
4991     JNJ  2017-12-01      

In [3]:
prices = data['Price'] 
data = data.drop(['Company', 'Time', 'Price'], axis = 1)
data

Total Assets  Total Liabilities  Total Equity  \
0          17311.4            13811.8        3499.6   
1          66221.0            28568.0       37653.0   
2          24916.0            11827.0       13089.0   
3           4243.1             2202.5        2040.6   
4          30387.7            28055.6        2332.1   
5          14752.0             4900.0        9852.0   
6           6530.0             4119.0        2411.0   
7          18717.9            15360.8        3357.0   
8          14493.5             5907.3        8586.1   
9          48275.1            39211.4        9063.7   
10          1149.4              553.7         595.7   
11          7929.2             7346.5         582.7   
12          9295.0             4402.0        4893.0   
13          3683.1             2522.5        1160.6   
14          3197.4             1552.5        1644.9   
15        346808.0           166313.0      180495.0   
16          4598.9             2909.6        1689.3   
17         13951.8             8294.4        5657.4   
18         25110.0            15469.0        9641.0   
19         70256.0            50428.0       19828.0   
20          1468.3              185.8        1282.5   
21          9231.4             9722.8        -491.4   
22          6202.8             3449.2        2753.6   
23         27743.0            20673.0        7070.0   
24          1532.0              986.7         545.4   
25         32666.0            18809.0       13857.0   
26          2872.5             1869.3        1003.2   
27          2230.9              788.0        1442.9   
28         15849.1             9337.2        6511.9   
29         13505.9             7808.3        5697.6   
...            ...                ...           ...   
4970      200451.0           174834.0       25617.0   
4971        6181.5             3751.0        2430.5   
4972       24048.5            21670.7        2377.8   
4973         649.3              158.5         490.8   
4974        2082.6              705.6        1377.0   
4975        2910.5              813.6        2096.9   
4976       44647.7            35072.7        9575.0   
4977       44003.0            45881.0       -1878.0   
4978        4144.2             3041.7        1102.4   
4979      260078.0           113356.0      146722.0   
4980        3068.1             1592.3        1475.7   
4981        2034.6              741.0        1293.6   
4982       45326.0            30949.0       14377.0   
4983        3260.9             2268.0         992.9   
4984        8482.3             5067.1        3415.2   
4985       19913.4            15015.9        4897.5   
4986        5778.9             4911.0         867.9   
4987       10855.2             5477.8        5377.4   
4988        9962.0             7031.0        2931.0   
4989       14104.7             9144.7        4960.0   
4990       15068.0            12038.0        3030.0   
4991      157303.0            97143.0       60160.0   
4992        5975.5             3390.6        2585.0   
4993       14605.0            10333.0        4272.0   
4994      444097.0           302090.0      142007.0   
4995       59521.3            49253.6       10267.7   
4996        2755.8             2021.5         734.3   
4997        1147.0              318.6         828.4   
4998        4093.2             2477.8        1615.4   
4999        1586.9             1007.5         579.4   

      Total Shares Out. on Filing Date  Book Value / Share  \
0                                125.6               16.95   
1                               1231.2               30.59   
2                                498.9               20.60   
3                                106.5               19.23   
4                                 53.0               43.67   
5                                987.6                8.61   
6                                124.7               19.26   
7                                166.2               20.04   
8                               2644.

In [4]:
col_list = []
for col in data.columns:
    col_list.append(col)
    
for col in col_list:
    print(col)
    data[col] = data[col].astype(float) # Converting columns to floats
    print("Done")

# Normalising the data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(data) 
data.loc[:,:] = scaled_values

data.insert(data.shape[1], 'Price', prices)
data

Total Assets
Done
Total Liabilities
Done
Total Equity
Done
Total Shares Out. on Filing Date
Done
Book Value / Share
Done
Tangible Book Value
Done
Tangible Book Value Per Share
Done
Total Debt
Done
Net Debt
Done
Total Revenues
Done
Operating Income
Done
Revenue Per Share
Done
Basic EPS
Done
Normalized Basic EPS
Done
Dividend Per Share
Done
EBITA
Done
EBIT
Done
Normalized Net Income
Done
Net Income
Done
Cash from Operations
Done
Cash from Investing
Done
Cash from Financing
Done
Net Change in Cash
Done
Levered Free Cash Flow
Done
Unlevered Free Cash Flow
Done
Free Cash Flow / Share
Done


Total Assets  Total Liabilities  Total Equity  \
0         0.019172           0.016582      0.034032   
1         0.073376           0.034300      0.111647   
2         0.027600           0.014198      0.055825   
3         0.004689           0.002642      0.030717   
4         0.033664           0.033685      0.031379   
5         0.016336           0.005881      0.048468   
6         0.007224           0.004943      0.031559   
7         0.020731           0.018442      0.033708   
8         0.016049           0.007091      0.045592   
9         0.053488           0.047080      0.046677   
10        0.001261           0.000662      0.027433   
11        0.008774           0.008819      0.027404   
12        0.010288           0.005283      0.037199   
13        0.004069           0.003026      0.028717   
14        0.003530           0.001862      0.029818   
15        0.384336           0.199693      0.436259   
16        0.005084           0.003491      0.029918   
17        0.015449           0.009957      0.038936   
18        0.027815           0.018571      0.047989   
19        0.077848           0.060548      0.071139   
20        0.001614           0.000221      0.028994   
21        0.010217           0.011672      0.024963   
22        0.006861           0.004139      0.032337   
23        0.030733           0.024820      0.042146   
24        0.001685           0.001182      0.027319   
25        0.036189           0.022582      0.057570   
26        0.003170           0.002242      0.028359   
27        0.002459           0.000944      0.029359   
28        0.017552           0.011209      0.040878   
29        0.014955           0.009373      0.039027   
...            ...                ...           ...   
4970      0.222136           0.209925      0.084295   
4971      0.006837           0.004501      0.031603   
4972      0.026639           0.026018      0.031483   
4973      0.000706           0.000188      0.027195   
4974      0.002295           0.000845      0.029209   
4975      0.003212           0.000974      0.030845   
4976      0.049468           0.042110      0.047839   
4977      0.048753           0.055088      0.021812   
4978      0.004580           0.003650      0.028585   
4979      0.288218           0.136107      0.359509   
4980      0.003387           0.001909      0.029433   
4981      0.002242           0.000887      0.029019   
4982      0.050219           0.037159      0.058752   
4983      0.003601           0.002721      0.028336   
4984      0.009387           0.006082      0.033841   
4985      0.022056           0.018027      0.037209   
4986      0.006391           0.005894      0.028052   
4987      0.012017           0.006575      0.038300   
4988      0.011027           0.008440      0.032740   
4989      0.015618           0.010978      0.037351   
4990      0.016686           0.014452      0.032965   
4991      0.174318           0.116639      0.162795   
4992      0.006609           0.004069      0.031954   
4993      0.016173           0.012405      0.035788   
4994      0.492156           0.362724      0.348794   
4995      0.065951           0.059137      0.049413   
4996      0.003041           0.002425      0.027748   
4997      0.001258           0.000380      0.027962   
4998      0.004523           0.002973      0.029751   
4999      0.001745           0.001207      0.027396   

      Total Shares Out. on Filing Date  Book Value / Share  \
0                             0.001569            0.000400   
1                             0.015476            0.000453   
2                             0.006265            0.000414   
3                             0.001328            0.000409   
4                             0.000655            0.000503   
5                             0.012412            0.000369   
6                             0.001557            0.000409   
7                             0.002079            0.000412   
8                             0.03325

In [5]:
# Converting Data to Numpy Arrays
NpMatrix = data.to_numpy(dtype = None, copy = False)
X = NpMatrix[:,0:26] # Parameters
y = NpMatrix[:,26] # Price
print("X:", X)
print("\n")
print("Type X:", type(X))
print("Length of Individual X_train Vector:", len(X[1]))
print("Total Number of Training instances:", len(X))
print("\n")
print(y)
print("\n")
print("Type y:", type(y))
print("Length of Individual y_train vector", y[0])
print("Total number of y values", len(y))
number_of_features = len(X[1])

X: [[1.91721365e-02 1.65816374e-02 3.40324380e-02 ... 4.83453945e-01
  4.77627905e-01 4.27248134e-03]
 [7.33761036e-02 3.42997600e-02 1.11646922e-01 ... 4.80980464e-01
  4.75545551e-01 3.90452105e-03]
 [2.75999194e-02 1.41984405e-02 5.58245974e-02 ... 4.62101128e-01
  4.55867310e-01 3.76725837e-03]
 ...
 [1.25797232e-03 3.80029127e-04 2.79620670e-02 ... 4.58195791e-01
  4.50900643e-01 3.74751181e-03]
 [4.52309262e-03 2.97263226e-03 2.97505448e-02 ... 4.58272724e-01
  4.51238899e-01 3.81734721e-03]
 [1.74549062e-03 1.20720785e-03 2.73962081e-02 ... 4.57048890e-01
  4.49811725e-01 3.79471091e-03]]


Type X: <class 'numpy.ndarray'>
Length of Individual X_train Vector: 26
Total Number of Training instances: 5000


[75.03 48.35 18.44 ...  8.71 47.18 33.97]


Type y: <class 'numpy.ndarray'>
Length of Individual y_train vector 75.03
Total number of y values 5000


In [6]:
#Loss functions
# https://keras.io/api/losses/regression_losses/#meanabsoluteerror-class
# https://keras.io/api/losses/

# Create function returning a compiled network
def create_network():
    
    # Start neural network
    network = models.Sequential()

    # Add fully connected layer with a ReLU activation function
    # Layer 1
    network.add(layers.Dense(units=26, activation='relu', input_shape=(number_of_features,)))
    
     #Add Dropout Layer
    network.add(layers.Dropout(0.3))
    
    #Layer 2
    network.add(layers.Dense(units=14, activation='relu', input_shape=(number_of_features,)))
    
     #Add Dropout Layer
    network.add(layers.Dropout(0.2))
    
    #Layer 3
    network.add(layers.Dense(units=8, activation='relu', input_shape=(number_of_features,)))
    
     #Add Dropout Layer
    network.add(layers.Dropout(0.1))
    
    #Layer 4
    network.add(layers.Dense(units=3, activation='relu', input_shape=(number_of_features,)))
    
     #Add Dropout Layer
    network.add(layers.Dropout(0.05))

    # Add fully connected layer with a ReLU activation function
    network.add(layers.Dense(units=1, activation='relu'))

    # Compile neural network
    network.compile(#loss='mse', # Mean Squared Error
                    loss = MeanAbsoluteError(),
                    optimizer='adam', # Root Mean Square Propagation
                    metrics=['mae', 'mse']) # Accuracy performance metric
    
    # Return compiled network
    return network

In [7]:
neural_network = KerasRegressor(build_fn=create_network, 
                                 epochs= 2000, 
                                 batch_size=6)

In [10]:
# Evaluate neural network using three-fold cross-validation
scores = cross_val_score(neural_network, X, y, cv=5)

Train on 4000 samples
Epoch 1/2000
4000/4000 [==============================] - 1s 337us/sample - loss: 59.9867 - mae: 59.9867 - mse: 21850.9121
Epoch 2/2000
4000/4000 [==============================] - 1s 217us/sample - loss: 53.2635 - mae: 53.2635 - mse: 20528.6602
Epoch 3/2000
4000/4000 [==============================] - 1s 230us/sample - loss: 51.5530 - mae: 51.5530 - mse: 20236.7812
Epoch 4/2000
4000/4000 [==============================] - 1s 214us/sample - loss: 49.7535 - mae: 49.7535 - mse: 20071.7773
Epoch 5/2000
4000/4000 [==============================] - 1s 234us/sample - loss: 48.5295 - mae: 48.5295 - mse: 19967.6816
Epoch 6/2000
4000/4000 [==============================] - 1s 245us/sample - loss: 47.7408 - mae: 47.7408 - mse: 19815.6445
Epoch 7/2000
4000/4000 [==============================] - 1s 241us/sample - loss: 47.3480 - mae: 47.3480 - mse: 19940.2949
Epoch 8/2000
4000/4000 [==============================] - 1s 211us/sample - loss: 46.8927 - mae: 46.8927 - mse: 19901

Epoch 67/2000
4000/4000 [==============================] - 1s 308us/sample - loss: 42.7190 - mae: 42.7190 - mse: 18991.0430
Epoch 68/2000
4000/4000 [==============================] - 1s 331us/sample - loss: 43.0417 - mae: 43.0417 - mse: 19092.8906
Epoch 69/2000
4000/4000 [==============================] - 1s 303us/sample - loss: 43.3015 - mae: 43.3015 - mse: 19140.2207
Epoch 70/2000
4000/4000 [==============================] - 1s 313us/sample - loss: 43.0413 - mae: 43.0413 - mse: 19099.4883
Epoch 71/2000
4000/4000 [==============================] - 1s 301us/sample - loss: 42.3062 - mae: 42.3062 - mse: 18966.5430
Epoch 72/2000
4000/4000 [==============================] - 1s 318us/sample - loss: 42.5488 - mae: 42.5489 - mse: 18975.7773 1s - loss: 60.8340 - mae: 60.8
Epoch 73/2000
4000/4000 [==============================] - 1s 314us/sample - loss: 42.7750 - mae: 42.7750 - mse: 18845.1152
Epoch 74/2000
4000/4000 [==============================] - 1s 299us/sample - loss: 42.2095 - mae: 42.

4000/4000 [==============================] - 1s 319us/sample - loss: 40.0738 - mae: 40.0738 - mse: 17340.6016
Epoch 133/2000
4000/4000 [==============================] - 1s 322us/sample - loss: 39.4578 - mae: 39.4578 - mse: 17331.2617
Epoch 134/2000
4000/4000 [==============================] - 1s 318us/sample - loss: 39.4443 - mae: 39.4443 - mse: 17275.9141
Epoch 135/2000
4000/4000 [==============================] - 1s 281us/sample - loss: 39.3466 - mae: 39.3466 - mse: 17134.7949
Epoch 136/2000
4000/4000 [==============================] - 2s 394us/sample - loss: 39.5978 - mae: 39.5978 - mse: 17063.9434
Epoch 137/2000
4000/4000 [==============================] - 1s 325us/sample - loss: 39.9381 - mae: 39.9381 - mse: 17385.2070
Epoch 138/2000
4000/4000 [==============================] - 1s 365us/sample - loss: 39.0100 - mae: 39.0100 - mse: 16990.5781
Epoch 139/2000
4000/4000 [==============================] - 1s 299us/sample - loss: 39.0678 - mae: 39.0678 - mse: 16836.3926
Epoch 140/2000


4000/4000 [==============================] - 1s 286us/sample - loss: 38.6491 - mae: 38.6491 - mse: 15434.4639
Epoch 198/2000
4000/4000 [==============================] - 1s 308us/sample - loss: 38.1496 - mae: 38.1496 - mse: 15749.3516
Epoch 199/2000
4000/4000 [==============================] - 1s 299us/sample - loss: 38.8151 - mae: 38.8151 - mse: 15637.6641
Epoch 200/2000
4000/4000 [==============================] - 1s 352us/sample - loss: 38.1206 - mae: 38.1207 - mse: 15511.0244
Epoch 201/2000
4000/4000 [==============================] - 1s 286us/sample - loss: 38.4657 - mae: 38.4657 - mse: 16147.6992
Epoch 202/2000
4000/4000 [==============================] - 1s 336us/sample - loss: 38.9884 - mae: 38.9884 - mse: 16043.1650
Epoch 203/2000
4000/4000 [==============================] - 1s 303us/sample - loss: 38.6009 - mae: 38.6009 - mse: 15492.0615
Epoch 204/2000
4000/4000 [==============================] - 1s 290us/sample - loss: 38.1167 - mae: 38.1167 - mse: 15767.6016
Epoch 205/2000


KeyboardInterrupt: 

In [ ]:
print(scores)
print("Mean MAE:", scores.mean())
print("Standard Deviation of the mean:", scores.std())